In [ ]:
from vertexai import agent_engines


# -----------------------------
# Constants & Display Messages
# -----------------------------
ENGINE_RESOURCE = "projects/641879769713/locations/us-central1/reasoningEngines/4165059997178265600"
DEFAULT_USER = "amirmeimand"




"""
Initialize the Vertex AI Agent Engine and create a session only once.
Returns (adk_app, user, session_id)
"""
adk_app = agent_engines.get(ENGINE_RESOURCE)
user = DEFAULT_USER

# Create a session (idempotent enough for our needs); then reuse the latest.
adk_app.create_session(user_id=user)
sessions = adk_app.list_sessions(user_id=user)
session_id = sessions["sessions"][0]["id"]



In [ ]:
for event in adk_app.stream_query(
        user_id=user,
        session_id=session_id,
        message=info_text,
    ):
        parts = event.get("content", {}).get("parts", []) if isinstance(event, dict) else []
        text = parts[0].get("text") if parts and isinstance(parts[0], dict) else None

        if text and isinstance(text, str) and text.strip():
            # Choose COMPLETE label
            if last_start_idx_printed in COMPLETE_AFTER_START:
                c_idx = COMPLETE_AFTER_START[last_start_idx_printed]
            else:
                c_idx = min(complete_idx, len(MESSAGES_COMPLETE) - 1)

            # Only print if we haven’t already passed the last COMPLETE
            if complete_idx < len(MESSAGES_COMPLETE):
                log_area.markdown(MESSAGES_COMPLETE[c_idx])
                with log_area.expander(f"Model output (step {c_idx + 1})", expanded=False):
                    st.write(text)

            complete_idx = max(complete_idx, c_idx) + 1
            still_working_logged = False

            # Advance START unless we’re at the last one
            if (last_start_idx_printed or 0) + 1 < len(MESSAGES_START):
                start_idx = last_start_idx_printed + 1
                log_area.markdown(MESSAGES_START[start_idx])                
                last_start_idx_printed = start_idx

        else:
            if not still_working_logged:
                log_area.write("Still working..")
                still_working_logged = True
